In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import classification_report


In [2]:
wine_base = pd.read_csv("Our_dataset/StemmedWord2vecTop3_parsed_weather_labeled.csv", index_col=0)
wine_base = wine_base.reset_index()
wine_base= wine_base[pd.notnull(wine_base['description'])]
wine_base

,vintage,country,description,points,price,province,region_1,taster_name,variety,winery,...,pr_5,pr_6,pr_7,pr_8,pr_9,tas_5,tas_6,tas_7,tas_8,tas_9
0,1952,8,year ii oak deep oldgold color rich sweet wood...,95,499.0,164,1201,13,376,10276,...,120.1810,54.8968,13.36560,12.36770,71.64850,16.0018,20.1771,21.6355,21.3967,18.5723
1,1952,8,astonish age rich toffe warm intens concentr d...,96,415.0,164,1201,13,376,1703,...,120.1810,54.8968,13.36560,12.36770,71.64850,16.0018,20.1771,21.6355,21.3967,18.5723
2,1957,8,great balanc fresh age vener port cours mani w...,95,375.0,164,1201,13,376,8743,...,92.0486,44.2920,6.72316,7.40562,49.36750,15.0959,18.7823,22.4479,22.4856,20.6839
3,1961,8,old gold color matur colheita great vintag por...,95,320.0,164,1201,13,376,5078,...,90.5883,43.2681,25.67690,2.98664,43.91890,18.0467,20.0059,21.9887,23.7509,20.2514
4,1963,8,matur port past year rich fig chocol touch alm...,95,467.0,164,1201,17,376,6238,...,38.1876,48.3052,3.52590,4.78859,29.14870,16.1193,18.4244,22.1413,20.8027,19.1788
5,1963,8,fabul legendari vintag port year age wood conc...,96,400.0,164,1201,13,376,10276,...,38.1876,48.3052,3.52590,4.78859,29.14870,16.1193,18.4244,22.1413,20.8027,19.1788
6,1963,8,sweet soft age light wood spice edg spirit fre...,87,790.0,164,1201,13,376,1703,...,38.1876,48.3052,3.52590,4.78859,29.14870,16.1193,18.4244,22.1413,20.8027,19.1788
7,1964,8,toffe vanilla sweet molass caramel consist hon...,91,125.0,164,1201,13,376,15321,...,22.0317,56.3338,8.25345,5.69519,38.65200,18.4907,19.7043,22.1509,22.5522,21.9118
8,1964,8,old gold color singl year cask wonder velvet t...,95,588.0,164,1201,13,376,13756,...,22.0317,56.3338,8.25345,5.69519,38.65200,18.4907,19.7043,22.1509,22.5522,21.9118
9,1965,8,year singl quinta vintag port concentr toffe r...,94,240.0,164,1201,13,376,8743,...,17.4721,16.5924,4.11930,3.63757,111.79200,18.1501,20.2796,20.0903,22.2992,17.8837


In [ ]:
#fig, ax = plt.subplots(figsize = (12, 5))
#points = Y.astype(int)
#sns.distplot(points,hist=True,bins = 20,hist_kws={'edgecolor':'black'})

In [3]:
Y = wine_base['points'].copy()
#DECIDE NUMBER OF BINS 
bins = 5
#labels=["very_low", "low", "medium","high","very_high"]
#labels=["low","medium","high"]
Y = pd.cut(Y,bins)#labels = labels)  

In [4]:
Y

0          (92.0, 96.0]
1          (92.0, 96.0]
2          (92.0, 96.0]
3          (92.0, 96.0]
4          (92.0, 96.0]
5          (92.0, 96.0]
6          (84.0, 88.0]
7          (88.0, 92.0]
8          (92.0, 96.0]
9          (92.0, 96.0]
10         (92.0, 96.0]
11         (92.0, 96.0]
12         (92.0, 96.0]
13         (92.0, 96.0]
14        (96.0, 100.0]
15         (92.0, 96.0]
16         (92.0, 96.0]
17         (84.0, 88.0]
18         (92.0, 96.0]
19        (79.98, 84.0]
20         (92.0, 96.0]
21         (92.0, 96.0]
22         (92.0, 96.0]
23         (88.0, 92.0]
24         (88.0, 92.0]
25         (92.0, 96.0]
26         (92.0, 96.0]
27         (88.0, 92.0]
28         (92.0, 96.0]
29         (88.0, 92.0]
              ...      
110005     (88.0, 92.0]
110006     (88.0, 92.0]
110007     (88.0, 92.0]
110008     (88.0, 92.0]
110009     (88.0, 92.0]
110010     (88.0, 92.0]
110011     (88.0, 92.0]
110012     (88.0, 92.0]
110013     (88.0, 92.0]
110014     (88.0, 92.0]
110015     (84.0

In [ ]:
basic = ['vintage', 'country', 'price', 'province', 'region_1', 'taster_name', 'variety','winery']
word = ["word_count"]
tfGroup = ['tf_grouped_1','tf_grouped_2', 'tf_grouped_3']
tfIdfGroup = ['tfIdf_grouped_1', 'tfIdf_grouped_2', 'tfIdf_grouped_3']
tfFull = ['tf_fullData_1', 'tf_fullData_2', 'tf_fullData_3',]
tfIdfFull = ['tfIdf_fullData_1', 'tfIdf_fullData_2', 'tfIdf_fullData_3']
weather = ['pr_5', 'pr_6', 'pr_7', 'pr_8', 'pr_9', 'tas_5', 'tas_6', 'tas_7', 'tas_8', 'tas_9']
word2vec = ['similarityTop3WinesByVariety']

In [ ]:
features = basic + word + weather + word2vec 
X = wine_base.loc[:,features]
X.columns

In [ ]:
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=42)
classifier = DecisionTreeClassifier()
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)  
acc = accuracy_score(y_test,y_pred)
weightedPrec = precision_score(y_test,y_pred,average="weighted")
#get feature importances
lista = []
for name, importance in zip(features, classifier.feature_importances_):
    lista.append([name, importance])

In [ ]:
file = open("classification" + str(bins) + "bins.txt","a")
file.write("Decision Tree Classifier with points divided into " + str(bins) + " bins \n")
file.write("Built on: " + str(features) + "\n")
file.write("Test size: " + str(test_size) + "\n")
file.write("   " +"accuracy " + str(acc) + "\n")
file.write("   " +"weightedPrec " + str(weightedPrec) + "\n")
file.write("Feature importances: \n")
for el in lista: 
    file.write("   " + el[0] + ":   " + str(el[1]) + "\n")
file.write("Report By predicted class: \n")
file.write(classification_report(y_test, y_pred, target_names=labels))
file.close()

In [ ]:
with open("classification5bins.txt", 'r+') as f:
        line =" [(79.98, 84.0] < (84.0, 88.0] < (88.0, 92.0] < (92.0, 96.0] < (96.0, 100.0]]"
        content = f.read()
        f.seek(0, 0)
        f.write(line.rstrip('\r\n') + '\n' + content)